In [6]:
# This program draws;
# the relation between
#    (X) Stock/Flow ratio, and
#    (Z) the price of gold

In [7]:
# references
# Gold
# [1] https://www.usgs.gov/centers/national-minerals-information-center/gold-statistics-and-information
# [2] https://www.gold.org/goldhub/data/how-much-gold
# [3] https://www.macrotrends.net/1333/historical-gold-prices-100-year-chart

above_ground_2022 = 208_874 # tonnes

# Silver
# [4] https://cpmgroup.com/how-much-silver-is-above-ground/
# [5] https://companiesmarketcap.com/silver/marketcap/#:~:text=Estimated%20Market%20Cap%3A%20%241.310%20T
# Platinum
# [6] https://swissgoldsafe.ch/en/additional-information/fundamentals-precious-metals/stock-to-flow-ratio/#:~:text=Compared%20to%20other%20precious%20metals,for%20silver%20stands%20at%2022.
# [7] https://companiesmarketcap.com/platinum/marketcap/
# Palladium
# [8] https://www.goldpriceforecast.com/explanations/size-the-palladium-market/
# [9] https://companiesmarketcap.com/palladium/marketcap/

# silver 30~60 - $1.310T
# platinum 1 - $229.82B
# palladium 1 - $217.61B

In [8]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

Stock/Flow - Gold

In [9]:
df_gold_q = pd.read_csv('../input/20230709_gold_production.csv')
df_gold_q['Flow'] = df_gold_q['Amount'].shift(-1)
df_gold_q['Stock'] = above_ground_2022 - df_gold_q.loc[::-1, 'Flow'].fillna(0).cumsum()[::-1]

df_gold_q['Stock_to_Flow'] = df_gold_q['Stock']/df_gold_q['Flow']

Price

In [10]:
df_gold_p = pd.read_csv('../input/20230709_gold_price.csv')
df_gold_p.rename(columns={'Year Close':'Price'},inplace=True)

Merge

In [11]:
df_gold = pd.merge(df_gold_q,df_gold_p,how='left',on='Year')
df_gold['market_value'] = df_gold['Stock']*35274*df_gold['Price']

Bitcoin

In [12]:
df_bitcoin = pd.read_csv('../input/20230710_bitcoin_s2f.csv',index_col=0)
df_bitcoin['market_value'] = df_bitcoin['Stock']*df_bitcoin['Price']

Draw

In [13]:
plt.figure(figsize=(20,15))
plt.rcParams['font.size'] = 20
plt.rcParams['axes.axisbelow'] = True
plt.style.use('dark_background')

# plt.xscale('log')
plt.yscale('log')
x_min,x_max = 0,80
y_min,y_max = 5,14
plt.xlim(x_min,x_max)
plt.ylim(10**y_min,10**y_max)

xticks=[i for i in range(x_min,x_max+1) if i%10==0]
xlabels = [i for i in range(x_min,x_max+1) if i%10==0]
plt.xticks(ticks=xticks, labels=xlabels)
yticks=[10**i for i in range(y_min,y_max+1)]
_str = ['','K','M','B','T']
ylabels = [f'${10**(i%3):,}{_str[i//3]}' for i in range(y_min,y_max+1)]
plt.yticks(ticks=yticks, labels=ylabels)

for i in xticks:
    plt.axvline(x=i,color="grey",linewidth=1)
for i in yticks:
    plt.axhline(y=i,color="grey",linewidth=1)

other_metals = [
    ['Silver',  30,   1.310*10**12,1.0],
    ['Platinum', 1, 229.82 *10**9, 1.2],
    ['Palladium',1, 217.61 *10**9, 0.8],
]
for name,x,y,_y in other_metals:
    plt.scatter(x,y,s=10**3,color='grey')
    plt.text(x+2,y*_y,name,color='white',fontsize=15)

plt.scatter(df_gold['Stock_to_Flow'],df_gold['market_value'],s=10**2,color='yellow')
plt.scatter(df_bitcoin['Stock_to_Flow'],df_bitcoin['market_value'],s=10**1,color='white')
plt.text(70,10**13,'Gold',color='yellow',fontsize=15)
plt.text(70,10**11.75,'Bitcoin',color='white',fontsize=15)

plt.title(f"Stock/Flow and Market Capitalization")
plt.xlabel("Stock/Flow")
plt.ylabel("Market Capitalization")

plt.savefig("../output/20230710_Stock_to_Flow_ylog.png")
plt.close()

In [14]:
plt.figure(figsize=(20,15))
plt.rcParams['font.size'] = 20
plt.rcParams['axes.axisbelow'] = True
plt.style.use('dark_background')

plt.xscale('log')
plt.yscale('log')
x_min,x_max = 1,80
y_min,y_max = 5,14
plt.xlim(x_min,x_max)
plt.ylim(10**y_min,10**y_max)

xticks=[i for i in range(x_min,x_max+1) if i%10==0 or i==1]
xlabels = [i for i in range(x_min,x_max+1) if i%10==0 or i==1]
plt.xticks(ticks=xticks, labels=xlabels)
yticks=[10**i for i in range(y_min,y_max+1)]
_str = ['','K','M','B','T']
ylabels = [f'${10**(i%3):,}{_str[i//3]}' for i in range(y_min,y_max+1)]
plt.yticks(ticks=yticks, labels=ylabels)

for i in xticks:
    plt.axvline(x=i,color="grey",linewidth=1)
for i in yticks:
    plt.axhline(y=i,color="grey",linewidth=1)

other_metals = [
    ['Silver',  30,   1.310*10**12,1.0],
    ['Platinum', 1, 229.82 *10**9, 1.2],
    ['Palladium',1, 217.61 *10**9, 0.8],
]
for name,x,y,_y in other_metals:
    plt.scatter(x,y,s=10**3,color='grey')
    plt.text(x*1.2,y*_y,name,color='white',fontsize=15)

plt.scatter(df_gold['Stock_to_Flow'],df_gold['market_value'],s=10**2,color='yellow')
plt.scatter(df_bitcoin['Stock_to_Flow'],df_bitcoin['market_value'],s=10**1,color='white')
plt.text(70,10**13,'Gold',color='yellow',fontsize=15)
plt.text(70,10**11.75,'Bitcoin',color='white',fontsize=15)

plt.title(f"Stock/Flow and Market Capitalization")
plt.xlabel("Stock/Flow")
plt.ylabel("Market Capitalization")

plt.savefig("../output/20230710_Stock_to_Flow_xlog_ylog.png")
plt.close()